

# Mohammed Mynuddin
# 950446781



# Simple automatic differentiation illustration

# Modify the attached python notebook for the automatic diffrentiation to include two more oprearators:

# Subtraction (x - y)
# Division (x / y)

In [180]:
from typing import Union, List

import numpy as np

np.set_printoptions(precision=4)

In [181]:
Numberable = Union[float, int]

def ensure_number(num: Numberable):
    if isinstance(num, NumberWithGrad):
        return num
    else:
        return NumberWithGrad(num)        

class NumberWithGrad(object):
    
    def __init__(self, 
                 num: Numberable,
                 depends_on: List[Numberable] = None,
                 creation_op: str = ''):
        self.num = num
        self.grad = None
        self.depends_on = depends_on or []
        self.creation_op = creation_op

    def __add__(self, 
                other: Numberable):
        return NumberWithGrad(self.num + ensure_number(other).num,
                              depends_on = [self, ensure_number(other)],
                              creation_op = 'add')
    
    def __mul__(self,
                other: Numberable = None):

        return NumberWithGrad(self.num * ensure_number(other).num,
                              depends_on = [self, ensure_number(other)],
                              creation_op = 'mul')
    def __sub__(self, 
                other: Numberable):
        return NumberWithGrad(self.num - ensure_number(other).num,
                              depends_on = [self, ensure_number(other)],
                              creation_op = 'sub')
    
    def __truediv__(self,
                other: Numberable = None):

        return NumberWithGrad(self.num / ensure_number(other).num,
                              depends_on = [self, ensure_number(other)],
                              creation_op = 'truediv')
    
    def backward(self, backward_grad: Numberable = None):
        if backward_grad is None: # first time calling backward
            self.grad = 1
        else: 
            # These lines allow gradients to accumulate.
            # If the gradient doesn't exist yet, simply set it equal
            # to backward_grad
            if self.grad is None:
                self.grad = backward_grad
            # Otherwise, simply add backward_grad to the existing gradient
            else:
                self.grad += backward_grad
        
        if self.creation_op == "add":
            # Simply send backward self.grad, since increasing either of these 
            # elements will increase the output by that same amount
            self.depends_on[0].backward(self.grad)
            self.depends_on[1].backward(self.grad)    

        if self.creation_op == "mul":

            # Calculate the derivative with respect to the first element
            new = self.depends_on[1] * self.grad
            # Send backward the derivative with respect to that element
            self.depends_on[0].backward(new.num)

            # Calculate the derivative with respect to the second element
            new = self.depends_on[0] * self.grad
            # Send backward the derivative with respect to that element
            self.depends_on[1].backward(new.num)
        if self.creation_op == "sub":
            # Simply send backward self.grad, since increasing either of these 
            # elements will increase the output by that same amount
            self.depends_on[0].backward(self.grad)
            self.depends_on[1].backward(self.grad*-1)    

        if self.creation_op == "truediv":

            # Calculate the derivative with respect to the first element
            new = self.depends_on[1]/(self.depends_on[1]*self.depends_on[1]) * self.grad
            # Send backward the derivative with respect to that element
            self.depends_on[0].backward(new.num)

            # Calculate the derivative with respect to the second element
            new = self.depends_on[0] /(self.depends_on[1]*self.depends_on[1])* self.grad
            # Send backward the derivative with respect to that element
            self.depends_on[1].backward(new.num*-1)

In [182]:
a = NumberWithGrad(3)

# Verifification
# Division 
# c= (4a+3)
# d= (a+2)
# subtraction s=c/d
# = (4a+3)/(a+2)
# derivative ds/da = 5 /( (a+2)*(a+2)) = 5/((3+2)*(3+2))=0.2

In [183]:
b = a * 4
c = b + 3
d = a + 2
e = c / d 
e.backward()

In [184]:
a.grad  # as expected  5/((3+2)*(3+2))=0.2

0.20000000000000007

# Verifification
# Subtraction
# c= (4a+3)
# d= (a+2)
# subtraction s=c-d
# = (4a+3)-(a+2)
# =4a+3-a-2
# =3a+1
# derivative ds/da = 3

In [185]:
x = NumberWithGrad(3)
y = x * 4
z = y + 3
m = x + 2
p = z-m 
p.backward()

In [186]:
x.grad  # as expected 3

3